In [ ]:
import os
import sys
import cv2
import csv
import locale
import sys
import rasterio
import numpy as np
import pandas as pd

from scipy.stats import mode
from PIL import Image
from rasterio.transform import from_origin
from datetime import datetime

Image.MAX_IMAGE_PIXELS = 933120000
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
ScoreThreshold = 0.5
#Tile size
CropSize = 300
model_type = "vit_h"
numberPoints = 32

parametresName = "S"+str(int(ScoreThreshold*100))+"_A"+str(CropSize)+bandOrder+model_type+"_PPS"+str(numberPoints)

# The script assumes that the bands are stored in GBR order
ImageRGB = np.array(cv2.imread('CO_RGB_8kmExt_8BIT.tif'),dtype=np.uint8)

# Conversion of the panel into an array of boolan values. Value 0 = null (NoData), value 1 = everything else.
converted_array = np.bool_(np.where(ImageRGB[:,:,0] == 0,0,1))

#Dimensions of the input image
x_length = ImageRGB.shape[0]
y_length = ImageRGB.shape[1]

#Final output
Multiband = (np.zeros((x_length, y_length),dtype=np.uint8))

step = int(CropSize/1)

#from segment_anything_hq import sam_model_registry, SamAutomaticMaskGenerator
from segment_anything import sam_model_registry,SamAutomaticMaskGenerator

if model_type == "vit_h":
  sam_checkpoint = "checkpoints/sam_vit_h_4b8939.pth"    # H
if model_type == "vit_b":
  sam_checkpoint = "checkpoints/sam_vit_b_01ec64.pth"  # B

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device="cuda")
mask_generator = SamAutomaticMaskGenerator(sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device="cuda"),points_per_side= numberPoints ,stability_score_thresh=ScoreThreshold, pred_iou_thresh=0.5)



In [ ]:
# get time
time = datetime.now()

# time format
formated_time = time.strftime("%A %H:%M")

# iterate over image
y = 0
while (y) < (y_length):

  x = 0
  while (x) < (x_length):

    if (y+CropSize)>y_length:
      dim_y = y_length
    else:
      dim_y = y+CropSize

    if (x+CropSize)>x_length:
      dim_x = x_length
    else:
      dim_x = x+CropSize

    # If there are too many nulls (more than a quarter) in the image, skip the iteration
    if np.sum(converted_array[x:dim_x, y:dim_y]) < (CropSize*CropSize/4):
      x = x+step
      continue
        
    Crop = ImageRGB[x:dim_x, y:dim_y, :]#.transpose(1,0,2)

    masks = mask_generator.generate(Crop) # GPU

    # promenna pro iterovani masek
    i = 0

    # iterating over masks
    while i < len(masks):

      a = np.array((masks[i]["segmentation"]),dtype= bool)
      b = np.uint8(((masks[i]["stability_score"])+0.005)*100)
      c = np.uint8(((masks[i]["predicted_iou"])+0.005)*100)
      d = np.uint16(np.sum(a))
      e = np.array(masks[i]["point_coords"])

      if i == 0:
        Objects = a
        SS = b
        IOU = c
        area = d
        coords = e
      else:
        Objects = np.dstack((Objects, a))
        SS = np.dstack((SS, b))
        IOU = np.dstack((IOU, c))
        area = np.dstack((area, d))
        coords = np.dstack((coords, e))

      i = i+1

    np.savez_compressed('objects/'+parametresName+'/'+str(x)+"_"+str(y)+'.npz', Objects=Objects, SS=SS,IOU= IOU,area= area, coords = coords)

    print("Column ",x)
    x = x+step

  time = datetime.now()
  print("Row ",y,"  ",str(time))
  y = y + step